In [ ]:
import babypandas as bpd
import numpy as np

# Lecture 6 –  More Querying and GroupBy
## DSC 10, Winter 2022

### Announcements

- Homework 1 is due on **Saturday 1/15 at 11:59pm**.
- Lab 2 is due on **Tuesday 1/18 at 11:59pm**.

### Agenda

- Recap: queries.
- Writing queries with multiple conditions.
- GroupBy.
- Challenge problems.

**Resources**: 
- [Resources tab of the course website](https://dsc10.com/resources/)
- [DSC 10 Reference Sheet](https://drive.google.com/file/d/1mQApk9Ovdi-QVqMgnNcq5dZcWucUKoG-/view)
- [BabyPandas Documentation](https://babypandas.readthedocs.io/en/latest/index.html)

## Recap: queries

### Recap: queries

- A "query" is code that extracts rows from a table for which certain condition(s) are true.
- Below we load in our NBA salaries 🏀 dataset from last time.

In [ ]:
salaries = bpd.read_csv('data/nba-2022.csv').set_index('Player')
salaries

### Question: Who was the highest paid center (C)?

In [ ]:
salaries

In [ ]:
salaries.get('Position') == 'C'

In [ ]:
centers_only = salaries[salaries.get('Position') == 'C']
centers_only

In [ ]:
centers_only.sort_values(by='Salary', ascending=False).index[0]

### Boolean indexing

To select only some rows of `salaries`:

1. Make a sequence (list/array/Series) of `True`s (keep) and `False`s (toss).
    - The values `True` and `False` are of the _Boolean_ data type.
2. Then pass it into `salaries[sequence_goes_here]`.

Rather than making the sequence by hand, we usually generate it by making a comparison.

### Elementwise comparisons

There are several types of comparisons we can make.

|symbol|meaning|
|--------|--------|
|`==` |equal to |
|`!=` |not equal to |
|`<`|less than|
|`<=`|less than or equal to|
|`>`|greater than|
|`>=`|greater than or equal to|

In [ ]:
# Find all players whose salary is at least 20 million
salaries[salaries.get('Salary') > 20000000]

## Queries with multiple conditions

### Multiple conditions

- To do a query with multiple conditions, use `&` for "and" and `|` for "or".
- **Must use parentheses around each query.**
- Note: Don't use the Python keywords `and` and `or` here! They do not behave as you'd want.
    - Read the course notes for an explanation.

In [ ]:
# Find all players who are guards (point guards or shooting guards)
salaries[(salaries.get('Position') == 'PG') | (salaries.get('Position') == 'SG')]

### The `&` and `|` operators work element-wise

In [ ]:
(salaries.get('Position') == 'PG')

In [ ]:
(salaries.get('Position') == 'SG')

In [ ]:
(salaries.get('Position') == 'PG') | (salaries.get('Position') == 'SG')

### Another example

Which Power Forwards (PF) on the Sacramento Kings have a salary of over 8 million?

In [ ]:
salaries[(salaries.get('Position') == 'PF') & 
         (salaries.get('Team') == 'Sacramento Kings') & 
         (salaries.get('Salary') > 8000000)]

### Discussion Question

Which of these queries evaluates to the number of centers **not** named Dwight Howard?

A. `salaries[(salaries.get('Position') == 'C') & (salaries.get('Player') != 'Dwight Howard')].shape[0]`

B. `salaries[(salaries.get('Position') == 'C') and (salaries.index != 'Dwight Howard')].shape[0]`

C. `salaries[(salaries.get('Position') == 'C') & (salaries.index != 'Dwight Howard')].shape[0]`

D. `salaries[(salaries.get('Position') == 'C') & (salaries.index != 'Dwight Howard')].shape[1]`

### To answer, go to **[menti.com](https://menti.com)** and enter the code **2745 1232**.

## GroupBy: Split, aggregate, combine

### Motivating Question: Which team had the highest payroll?

- We can find the payroll for any one team.
    - For the Warriors, `salaries[salaries.get('Team') == 'Golden State Warriors'].get('Salary').sum()`.
- But how can we find the payroll for **all** teams at once?

In [ ]:
salaries

In [ ]:
salaries[salaries.get('Team') == 'Golden State Warriors'].get('Salary').sum()

In [ ]:
salaries[salaries.get('Team') == 'Los Angeles Lakers'].get('Salary').sum()

In [ ]:
salaries[salaries.get('Team') == 'Brooklyn Nets'].get('Salary').sum()

It seems like there has to be a better way. And there is!

### `.groupby`

Observe what happens when we use the `.groupby` method on `salaries` with the argument `'Team'`:

In [ ]:
salaries.groupby('Team').sum()

### An illustrative example

Consider the DataFrame `pets` containing pet species, colors, and weights.

<div align=center>
<img src='images/pets.png' width=300>
</div>

### Step 1: Split

When we run `pets.groupby('Species').mean()`, `babypandas` internally creates three mini-DataFrames, one for each unique value of `'Species'`.

<div align=center>
<img src='images/dogs.png' width=300><img src='images/cats.png' width=300><img src='images/hamsters.png' width=300>
</div>

### Step 2: Aggregate

It then takes the `mean` of every column (other than `Species` itself) that it can:

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **0** | dog | black | 40 |
| **3** | dog | white | 80 |
| **4** | dog | golden | 25 |

<center>Mean Weight: (40 + 80 + 25) / 3 = 48.333</center>

---

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **1** | cat | golden | 15 |
| **2** | cat | black | 20 |

<center>Mean Weight: (15 + 20) / 2 = 17.5</center>

---

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **5** | hamster | golden | 1 |

<center>Mean Weight: (1) / 1 = 1</center>

### Step 3: Combine

Finally, it combines these means back into a single table, whose labels are equal to the unique values from the `Species` column, listed in alphabetical order.

<div align=center>
<img src='images/grouped.png' width=200>
</div>

These steps are split, aggregate, and combine.

### Let's try it out!

In [ ]:
pets = bpd.DataFrame().assign(
    Species=['dog', 'cat', 'cat', 'dog', 'dog', 'hamster'],
    Color=['black', 'golden', 'black', 'white', 'golden', 'golden'],
    Weight=[40, 15, 20, 80, 25, 1]
)

pets

In [ ]:
pets.groupby('Species').mean()

### Back to the NBA...

In [ ]:
salaries

In [ ]:
salaries.groupby('Team').sum()

### `.groupby` in general

1. Use `.groupby(column_name)` to gather rows which have the same value in the specified column (`column_name`).
    - In the previous example, we grouped by `'Team'`.
    - In the resulting DataFrame, there will be one row for every unique value of `'Team'`.
2. Apply an **aggregation** method **within** each group.
    - In the previous example, we applied the `sum` method to every `'Team'`.
    - The aggregation method is applied individually to each column.

Some aggregation method: `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, `.min()`.

### Observation #1

- The index has changed to team names.
- In general, the new row labels are the *group labels* (the thing that is used to determine if two rows were in same group).

In [ ]:
salaries

In [ ]:
salaries.groupby('Team').sum()

### Observation #2

- The `'Position'` and `'Team'` columns have disappeared. Why?

In [ ]:
salaries.groupby('Team').sum()

### Disappearing columns

- The aggregation method is applied to each column.
- If it doesn't make sense to apply it to a particular column, that column will disappear.
- Example: We can't sum strings, like in the `'Position'` column.
- Surprising example: we *can* `max()` strings. How?

In [ ]:
salaries.groupby('Team').max() # Can you guess how the max position is determined?

### Other aggregation methods

- Aggregation methods work on arrays/lists/Series and return a single number.
- The most common ones are `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, and `.min()`.


In [ ]:
salaries.groupby('Team').count()

In [ ]:
salaries.groupby('Team').mean()

In [ ]:
salaries.groupby('Team').max()

### Question: Which team had the highest payroll?

Strategy:
1. Group by `'Team'`.
2. Within each group, compute the total salary.
3. Sort by salary in descending order.
4. Take the name of the first team.

In [ ]:
salaries.groupby('Team').sum().sort_values(by='Salary', ascending=False).index[0]

### Question: How many players play each position?

In [ ]:
salaries.groupby('Position').count()

### Aside: Better column names?

To rename a column:

1. Add a new column with `.assign` containing the same values as the old column(s).
2. Drop the old column(s) with `.drop(columns=list_of_column_labels)`.

In [ ]:
pos_counts = salaries.groupby('Position').count()
pos_counts

In [ ]:
pos_counts.assign(Count=pos_counts.get('Team')).drop(columns=['Team', 'Salary'])

## Example: IMDB Dataset

### Note: walkthrough video

Note, we're likely not going to cover this entire section during lecture. Instead, we've recorded a video walkthrough of the remainder of lecture, and encourage you to watch it for extra practice on GroupBy. This section will be very helpful in preparing for Homework 2 and 3!

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('xg7rnjWnZ48')

In [ ]:
imdb = bpd.read_csv('data/imdb.csv').set_index('Title')
imdb

### Question: How many movies appear from each decade?

In [ ]:
imdb.groupby('Decade').count()

In [ ]:
imdb.groupby('Decade').count().plot(y='Year');

### Question: What was the highest rated movie of the 1990's?

Let's try and do this two different ways.

#### Without grouping

In [ ]:
imdb[imdb.get('Decade') == 1990].sort_values('Rating', ascending=False).index[0]

#### With grouping

In [ ]:
imdb.reset_index().groupby('Decade').max()

- It turns out that this method **does not** yield the correct answer. 
- When we use an aggregation function (e.g. `.max`), aggregation is done to each column individually. 
- While it's true that the highest rated movie from the 1990s has a rating of 9.2, that movie is **not** Unforgiven – instead, Unforgiven is the movie that's the latest in the alphabet among all movies from the 1990s.
- Grouping is not helpful here.

## Combining queries and groupby

### In other words, more challenging problems!

We won't cover these problems in class, and we won't provide the solutions to these problems in the lecture notebook. To access the solutions, you'll either need to
1. Watch [the video](https://www.youtube.com/watch?v=xg7rnjWnZ48) embedded at the start of this section, or
2. Check [this Campuswire post](https://campuswire.com/c/G6950E967/feed/78) on Monday

**Before accessing the solutions, make sure to try these problems on your own – they're great prep for homeworks and projects!**

### Question: How many years have more than 3 movies rated above 8.5?

In [ ]:
...

#### Aside: Using `.sum()` on a boolean array

In [ ]:
(good_movies_per_year.get('Votes') > 3).sum() # Gives count of how many trues

### Question: Out of the years with more than 3 movies, which had the highest average rating?

In [ ]:
...

### Question: Which year had the longest movie titles, on average?

**Hint:** Use `.str.len()` on the column or index that contains the names of the movies.

In [ ]:
...

### Question: What is the average rating of movies from years that had at least 3 movies in the list?

In [ ]:
...

## Summary

### Summary

- We can write queries that involve multiple conditions, as long as we:
    - Put parentheses around all conditions.
    - Separate conditions using `&` if you require all to be true, or `|` if you require at least one to be true.
- The method call `df.groupby(column_name).name_of_method()` **aggregates** all rows with the same value for `column_name` into a single row in the resulting table, according to the aggregation method `name_of_method`.
    - Aggregation methods we've seen include `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, and `.min()`.
- **Next time:** A picture is worth a 1000 words – it's time to visualize!